Load in basic libraries.

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from matplotlib.pyplot import *
import queue, pyaudio 
from scipy import signal
from numpy import *
%matplotlib inline

## Explore the python fft function

In [ ]:
# Show the help for the fft function
np.fft.fft?

In [ ]:
from __future__ import division
# Create sinusoidal signal
t = np.r_[0:2:(1/44100)]
sig = np.sin(2*np.pi*5000*t)

# Perform FFT and create frequency axis
sigfft = np.fft.fft(sig, 1024 )
freq = np.r_[0:1024] # note this is not quite correct
# the code below is interesting, but it is better to create your own frequency axis.
# freq = np.fft.fftfreq(1024, 1/44100)

# Plot the signal and its FFT
# Plot the signal 
plt.figure()
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
# stem plot
plt.stem(t[0:441]*1000,sig[0:441],use_line_collection=True)
# continuous plot
#plt.plot(t[0:441]*1000,sig[0:441])
plt.legend(("fs = 44.1 kHz",""))
plt.xlabel("time (ms)")
plt.ylabel("Amplitude")

# Plot the FFT of the signal
plt.figure()
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.plot(freq,abs(sigfft)/np.max(abs(sigfft)))
plt.xlabel("frequency (kHz)")
plt.ylabel("Normalized Amplitude")
plt.legend(("fs = 44.1 kHz",""))


## Things to explain and change
### 1. In the example above we used a stem plot. Please comment this out and uncomment the continuous plot. 
### 2. Explain briefly why the continuous plot is easier to interpret - despite the fact it is sampled and a stem plot should be ok.
### 3. Explain why 10 ms corresponds to 441 samples
### 4. The FFT plot has the wrong frequency axis. Please fix it by modifying the code above.
### 5. The FFT plot of a real signal is redundant - explain.
### 6. Modify the FFT plot so that we only show the frequencies up to the Nyquist frequency.
### 7. Numpy provides a FFT frequency function. Comment your frequency axis code and uncomment the freq axis defined using the numpy fftfreq function. Plot the FFT using the numpy freq axis. 
### 8. Explain very, very carefully what happened with the FFT plot using the numpy fftfreq function. 


## Define the play_audio function

In [ ]:
def play_audio( Q, p, fs , dev=None):
    # play_audio plays audio with sampling rate = fs
    # Q - A queue object from which to play
    # p   - pyAudio object
    # fs  - sampling rate
    # dev - device number
    
    # Example:
    # fs = 44100
    # p = pyaudio.PyAudio() #instantiate PyAudio
    # Q = Queue.queue()
    # Q.put(data)
    # Q.put("EOT") # when function gets EOT it will quit
    # play_audio( Q, p, fs,1 ) # play audio
    # p.terminate() # terminate pyAudio
    
    # open output stream
    ostream = p.open(format=pyaudio.paFloat32, channels=1, rate=int(fs),output=True,output_device_index=dev)
    # play audio
    while (1):
        data = Q.get()
        if data is "EOT" :
            break
        try:
            ostream.write( data.astype(np.float32).tostring() )
        except:
            break

## Sample Sinusoidal Signals
### Case 1: 5 kHz sampled with 72 kHz - this will be our reference

In [ ]:
from __future__ import division
fs = [72000, 24000, 12000, 8000, 6000]

# Case 1: 5 kHz sampled with 72 kHz
# Create signal
t1 = np.r_[0:2:(1/fs[0])]
sig1 = np.sin(2*np.pi*5000*t1)

# Perform FFT and create frequency axis
sig1fft = np.fft.fft(sig1, 1024 )
freq1 = np.r_[0:1024]*(fs[0]/1024)

# Plot the signal 
plt.figure()
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.plot(t1[0:720]*1000,sig1[0:720])
plt.legend(("fs = 72 kHz",""))
plt.xlabel("time (ms)")
plt.ylabel("Amplitude")

# Plot the FFT of the signal
plt.figure()
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.plot(freq1[0:513]/1000,abs(sig1fft[0:513])/np.max(abs(sig1fft[0:513])))
plt.xlabel("frequency (kHz)")
plt.ylabel("Normalized Amplitude")
plt.legend(("fs = 72 kHz",""))
plt.axis([0,10,0,1])
plt.xticks(np.arange(0,10+1,1))

## Play the reference signal

In [ ]:
Qout = queue.Queue()
Qout.put( sig1 );
Qout.put( "EOT" );
p = pyaudio.PyAudio()
play_audio(Qout,p,fs[0])
p.terminate()

## Repeat with different sample rates and compare to the reference
### Case 2: 5 kHz sample with 24 kHz

In [ ]:
from __future__ import division
fs = [72000, 24000, 12000, 8000, 6000]

# Case 1: 5 kHz sampled with 24 kHz
# Create signal
t2 = np.r_[0:2:(1/fs[1])]
sig2 = np.sin(2*np.pi*5000*t2)

# Perform FFT and create frequency axis
sig2fft = np.fft.fft(sig2, 1024 )
freq2 = np.r_[0:1024]*(fs[1]/1024)

# Plot new signal and compare with the reference
plt.figure()
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))

plt.plot(t1[0:720]*1000,sig1[0:720])
plt.plot(t2[0:240]*1000,sig2[0:240])
plt.legend(("fs = 72 kHz","fs = 24 kHz"))
plt.xlabel("time (ms)")
plt.ylabel("Amplitude")

# Plot FFT of new signal and compare with the reference
plt.figure()
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.plot(freq1[0:513]/1000,abs(sig1fft[0:513])/np.max(abs(sig1fft[0:513])))
plt.plot(freq2[0:513]/1000,abs(sig2fft[0:513])/np.max(abs(sig2fft[0:513])))
plt.xlabel("frequency (kHz)")
plt.ylabel("Normalized Amplitude")
plt.legend(("fs = 72 kHz","fs = 24 kHz"))
plt.axis([0,10,0,1])
plt.xticks(np.arange(0,10+1,1))


## Play the new signal and the reference signal

In [ ]:
# Reference
Qout = Queue.Queue()
Qout.put( sig1 );
Qout.put( "EOT" );
p = pyaudio.PyAudio()
play_audio(Qout,p,fs[0])
p.terminate()

In [ ]:
# New 
Qout = Queue.Queue()
Qout.put( sig2 );
Qout.put( "EOT" );
p = pyaudio.PyAudio()
play_audio(Qout,p,fs[1])
p.terminate()

# Your Turn
## Case 3:  5 kHz sampled at 12 kHz
### Add your code below similar to case 2. 
#### Generate the new signal, compute its FFT, plot the new signal with the reference signal, plot the fft of the signal and compare with the reference signal.
#### Play both the reference signal and the new signal. Provide a brief written explanation for what you observe.

## Case 4:  5 kHz sampled at 8 kHz
### Add your code below similar to case 2. 
#### Generate the new signal, compute its FFT, plot the new signal with the reference signal, plot the fft of the signal and compare with the reference signal.
#### Play both the reference signal and the new signal. Provide a brief written explanation for what you observe.

## Case 5:  5 kHz sampled at 6 kHz
### Add your code below similar to case 2. 
#### Generate the new signal, compute its FFT, plot the new signal with the reference signal, plot the fft of the signal and compare with the reference signal.
#### Play both the reference signal and the new signal. Provide a brief written explanation for what you observe.

# 2D FFT Exercise

In [ ]:
# Import functions and libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from scipy import signal
import matplotlib.image as mpimg
import matplotlib.pylab as pylab

%matplotlib inline
pylab.rcParams['figure.figsize'] = (20.0, 7.0)

## Display Image

In [ ]:
im = mpimg.imread('zelda.tif')
im = im[:,:,1]
f = plt.figure()
plt.imshow(im,cmap='gray')
print (im.shape)

## Compare the Magnitude and Phase Spectrum of an Image
Write python code to compute the 2D FFT of an image using np.fft.fft2(). For the magnitude spectrum, please compute the log spectrum (20 x log(abs())). Please make sure you apply the np.fft.fftshift() function.

In [ ]:
# Put your code here
im_fft = 
fshift = 
magnitude_spectrum = 
phase_spectrum = 

# Make the plot
plt.subplot(131),plt.imshow(im, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(phase_spectrum, cmap = 'gray')
plt.title('Phase Spectrum'), plt.xticks([]), plt.yticks([])
plt.show() 

Which spectrum seems to contain more information? Put your answer here.

# JPEG DCT Demo
Please skim Chapter 8, Section 8.

## Define 2D DCT and IDCT

The DTC2 Transform is defined as follows:
$$ X[k] = 2\sum_{n=0}^{N-1}x[n]\cos\left(\frac{\pi k(2n+1)}{2N}\right),\ \ 0\le k \le N-1$$
$$ x[n] = \frac{1}{N}\sum_{k=0}^{N-1}\beta[k]X[k]\cos\left(\frac{\pi k (2n+1)}{2N}\right),\ \ 0 \le n \le N-1 $$
where $\beta[k]$ is a weighting function give by:
$$\beta[k] = \left\{ \begin{array}{c}
                     \frac{1}{2}, & \ \ k=0\\
                     1, & \ \ 1 \le k \le N-1\\
                     \end{array} \right. $$


In [ ]:
def dct2(a):
    return scipy.fft.dct( scipy.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return scipy.fft.idct( scipy.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

## Perform a blockwise DCT

In [ ]:
imsize = im.shape
dct = np.zeros(imsize)

# Do 8x8 DCT on image (in-place)
for i in r_[:imsize[0]:8]:
    for j in r_[:imsize[1]:8]:
        dct[i:(i+8),j:(j+8)] = dct2( im[i:(i+8),j:(j+8)] )

## Extract 8x8 block and look at its DCT coefficients

In [ ]:
pos = 150

# Extract a block from image
plt.figure()
plt.imshow(im[pos:pos+8,pos:pos+8],cmap='gray')
plt.title( "An 8x8 Image block")

# Display the dct of that block
plt.figure()
plt.imshow(dct[pos:pos+8,pos:pos+8],cmap='gray',vmax= np.max(dct)*0.01,vmin = 0, extent=[0,pi,pi,0])
plt.title( "An 8x8 DCT block")

## Display all DCT Blocks

In [ ]:
# Display entire DCT
plt.figure()
plt.imshow(dct,cmap='gray',vmax = np.max(dct)*0.01,vmin = 0)
plt.title( "8x8 DCTs of the image")

## Threshold DCT Coefficients

In [ ]:
# Threshold
thresh = 0.012
dct_thresh = dct * (abs(dct) > (thresh*np.max(dct)))
dct_thresh_discard = dct * (abs(dct) < (thresh*np.max(dct)))

plt.figure()
plt.imshow( np.hstack( (dct_thresh, dct_thresh_discard) ) ,cmap='gray',vmax = np.max(dct)*0.01,vmin = 0)
plt.title( "Thresholded 8x8 DCTs of the image: Left is Kept, Right is Discarded")

percent_nonzeros = np.sum( dct_thresh != 0.0 ) / (imsize[0]*imsize[1]*1.0)

print ("Keeping only %f%% of the DCT coefficients" % (percent_nonzeros*100.0))

## Compare DCT Compressed Image with Original

In [ ]:
im_dct = np.zeros(imsize)

for i in r_[:imsize[0]:8]:
    for j in r_[:imsize[1]:8]:
        im_dct[i:(i+8),j:(j+8)] = idct2( dct_thresh[i:(i+8),j:(j+8)] )
        
        
plt.figure()
plt.imshow( np.hstack( (im, im_dct) ) ,cmap='gray')
plt.title("Comparison between original and DCT compressed images" )

## Compare with DFT Compressed Image

Use the np.fft.fft2 to replace the dct2 function and repeat the above to compare the dft2 with the dct2 in terms of compression.

In [ ]:
dft = zeros(imsize,dtype='complex');
im_dft = zeros(imsize,dtype='complex');

# 8x8 DFT
# Please add your code here


# Apply a Threshold
thresh = 0.013
dft_thresh = dft * (abs(dft) > (thresh*np.max(abs(dft))))


percent_nonzeros_dft = np.sum( dft_thresh != 0.0 ) / (imsize[0]*imsize[1]*1.0)
print ("Keeping only %f%% of the DCT coefficients" % (percent_nonzeros*100.0))
print ("Keeping only %f%% of the DFT coefficients" % (percent_nonzeros_dft*100.0))

# 8x8 iDFT
# Please add your code here

        
        
plt.figure()
plt.imshow( np.hstack( (im, im_dct, abs(im_dft)) ) ,cmap='gray')
plt.title("Comparison between original, DCT compressed and DFT compressed images" )

# FFT DEMO
Please run through the demo below. There is no particular coding work required, but please answer the question at the end.

## Forward DFT:
$$X[k]=\sum_{n=0}^{N-1}x[n]W_N^{kn}$$ 
Recall from class that the DFT can be implemented as a matrix-vector multiplication. We know that the complexity of dense matrix-vector multiplication is  $\mathcal{O}(N^2)$ .

We can write this as  $\mathbf{X}=\mathbf{D}\mathbf{x}$ , where  $(\mathbf{D})kn=W_N^{kn}$.

In [ ]:
def W_N(N,k,n):
    return exp(-1j * 2 * np.pi * k * n / N)

def mydft(x):
    """Returns the 1D DFT of x using matrix-vector multiplication"""
    N = x.shape[0]
    n_idx = np.arange(N)
    k_idx = n_idx[:, None]
    return np.dot(W_N(N,k_idx,n_idx), x)

L = 1024
x = np.random.random(L) + 1j * np.random.random(L)

res = np.allclose(mydft(x), np.fft.fft(x))
if res:
    print ('DFT matrix matches built-in FFT')
else:
    print ('Error in DFT matrix multiplication')

%timeit mydft(x)
%timeit np.fft.fft(x)

## Symmetry in the DFT
Recall the periodicity/symmetry of the DFT:
$$ \begin{array}{ll}
   X[k+mN] & = &  \sum_{n=0}^{N-1}x[n]W_N^{(k+mN)n} \\
           & = & \sum_{n=0}^{N-1}x[n]W_N^{kn}\\
           & = & X[k]\\
   \end{array} $$     
where we have used the periodicity of $W_N$.

We can use this symmetry to split the DTF into two smaller DFTs:
$$ \begin{array}{ll}
    X[k] & = &  \sum_{n=0}^{N-1}x[n]W_N^{kn}\\
         & = & \sum_{m=0}^{N/2-1}x[2m]W_N^{k(2m)} + \sum_{m=0}^{N/2-1}x[2m+1]W_N^{k(2m+1)}\\
   \end{array} $$

The FFT uses this property to decompose the length-N DFT into two length-N/2 DFTs, repeadetly. This amounts to a complexity of $\mathcal{O}(N\log N)$.
   

In [ ]:
def myfft(x):
    """A recursive implementation of the 1D Cooley-Tukey FFT"""
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")
    elif N <= 32:  # this cutoff should be optimized
        return mydft(x)
    else:
        X_even = myfft(x[::2])
        X_odd = myfft(x[1::2])
        n_idx = np.arange(N)
        factor = W_N(N,n_idx, 1)
        return np.concatenate([X_even + factor[:N // 2] * X_odd,
          
                               X_even + factor[N // 2:] * X_odd])
    
res = np.allclose(myfft(x), np.fft.fft(x))
if res:
    print ('Our FFT matches built-in FFT')
else:
    print ('Error in recursive FFT implementation')
    
    
%timeit mydft(x)
%timeit myfft(x)
%timeit np.fft.fft(x)

## Leveraging Numpy Vectorization
To get the most out of Numpy (and MATLAB), it is best to use vectorized operations instead of FOR loops or recursioin. We can speed up our FFT by doing this.

In [ ]:
def myfft_vec(x):
    """A vectorized, non-recursive version of the Cooley-Tukey FFT"""
    N = x.shape[0]

    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")

    # N_min here is equivalent to the stopping condition above,
    # and should be a power of 2
    N_min = min(N, 32)
    
    # Perform an O[N^2] DFT on all length-N_min sub-problems at once
    n_idx = np.arange(N_min)
    k_idx = n_idx[:, None]
    D = W_N(N_min, n_idx, k_idx)
    X = np.dot(D, x.reshape((N_min, -1)))

    # build-up each level of the recursive calculation all at once
    while X.shape[0] < N:
        X_even = X[:, :X.shape[1] // 2]
        X_odd = X[:, X.shape[1] // 2:]
        factor = W_N(X.shape[0], np.arange(X.shape[0]), 0.5)[:, None]
        X = np.vstack([X_even + factor * X_odd,
                       X_even - factor * X_odd])

    return X.ravel()


res = np.allclose(myfft_vec(x), np.fft.fft(x))
if res:
    print ('Our vectorized FFT matches built-in FFT')
else:
    print ('Error in recursive FFT implementation')
    
%timeit mydft(x)
%timeit myfft(x)
%timeit myfft_vec(x)
%timeit np.fft.fft(x)

Please summarize your observations of the demo here: